#Chapter 22

Language Modeling toy example

i like this black cat.

prodict next word

i like this [] <= black


vocab: "100 words"

vocab
word -> index
i -> 2
like -> 45
this -> 30
black -> 55
cat -> 10
. -> 1

our goal is to prodict 
"i like this black cat []" <= .

input => i like this black cat
target => like this black cat.

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import sys


inputString = [2,45,30,55,10]
outputString = [45,30,55,10,1]

numFeatures = 100
vocabSize = 80


embeddings = []
for i in range(len(inputString)):
    x = np.random.randn(numFeatures,1)
    embeddings.append(x)


embeddings[0].shape
len(embeddings)

def getOneHot(idx):
    one_hot = np.zeros((vocabSize,1))
    one_hot[idx] = 1
    return one_hot


print(getOneHot(2))

[[0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [2]:
numUnits = 50
h0 = torch.tensor(np.zeros((numUnits,1)))
Wh = torch.tensor(np.random.uniform(0,1,(numUnits,numUnits)),requires_grad=True)
Wx = torch.tensor(np.random.uniform(0,1,(numUnits,numFeatures)),requires_grad=True)
Wy = torch.tensor(np.random.uniform(0,1,(vocabSize,numUnits)),requires_grad=True)

print(Wh.shape,Wx.shape,Wy.shape,h0.shape)

def stepForward(xt,Wx,Wh,Wy,prevMemory):
    x_frd = torch.matmul(Wx,torch.from_numpy(xt))
    h_frd = torch.matmul(Wh,prevMemory)
    ht = torch.tanh(x_frd+h_frd)
    yt_hat = F.softmax(torch.matmul(Wy,ht),dim=0)
    return ht,yt_hat

ht,yt_hat = stepForward(embeddings[0],Wx,Wh,Wy,h0)
ht.shape
yt_hat.shape
yt_hat.sum()




torch.Size([50, 50]) torch.Size([50, 100]) torch.Size([80, 50]) torch.Size([50, 1])


tensor(1.0000, dtype=torch.float64, grad_fn=<SumBackward0>)

In [3]:
def fullForwardRNN(X,Wx,Wh,Wy,prevMemory):
    y_hat = []
    for t in range(len(X)):
        ht,yt_hat = stepForward(X[t],Wx,Wh,Wy,prevMemory)
        prevMemory = ht
        y_hat.append(yt_hat)
    return y_hat  

y_hat = fullForwardRNN(embeddings,Wx,Wh,Wy,h0)

len(y_hat)
y_hat[0].shape


torch.Size([80, 1])

In [4]:
def computeLoss(y,y_hat):#y => 1 hope vectors , y_hat => prodictions of model
    loss = 0
    for yi,yi_hat in zip(y,y_hat):
        Li = -torch.log2(yi_hat[yi==1])
        loss += Li
    return loss/len(y)

y = []
for idx in outputString:
    y.append(getOneHot(idx))

print(computeLoss(y,y_hat))

def updateParams(Wx,Wh,Wy,dWx,dWh,dWy,lr):
    with torch.no_grad():
        Wx -= lr*dWx
        Wh -= lr*dWh
        Wy -= lr*dWy
    return Wx,Wh,Wy

def trainRNN(X,y,Wx,Wh,Wy,prevMemory,lr,nepoch):
    losses = []
    for epoch in range(nepoch):
        y_hat = fullForwardRNN(X,Wx,Wh,Wy,prevMemory)
        loss = computeLoss(y,y_hat)
        loss.backward()
        losses.append(loss)
        print("Loss after epoch=%d: %f" %(epoch,loss))
        sys.stdout.flush()
        dWx = Wx.grad.data
        dWh = Wh.grad.data
        dWy = Wy.grad.data
        Wx,Wh,Wy = updateParams(Wx,Wh,Wy,dWx,dWh,dWy,lr)
        Wx.grad.data.zero_()
        Wh.grad.data.zero_()
        Wy.grad.data.zero_()
    return Wx,Wh,Wy,losses

Wx,Wh,Wy,losses = trainRNN(embeddings,y,Wx,Wh,Wy,h0,0.001,100)

tensor([7.9592], dtype=torch.float64, grad_fn=<DivBackward0>)
Loss after epoch=0: 7.959230
Loss after epoch=1: 7.933272
Loss after epoch=2: 7.907358
Loss after epoch=3: 7.881489
Loss after epoch=4: 7.855665
Loss after epoch=5: 7.829885
Loss after epoch=6: 7.804150
Loss after epoch=7: 7.778461
Loss after epoch=8: 7.752816
Loss after epoch=9: 7.727217
Loss after epoch=10: 7.701663
Loss after epoch=11: 7.676154
Loss after epoch=12: 7.650692
Loss after epoch=13: 7.625276
Loss after epoch=14: 7.599906
Loss after epoch=15: 7.574583
Loss after epoch=16: 7.549308
Loss after epoch=17: 7.524079
Loss after epoch=18: 7.498899
Loss after epoch=19: 7.473767
Loss after epoch=20: 7.448683
Loss after epoch=21: 7.423649
Loss after epoch=22: 7.398664
Loss after epoch=23: 7.373730
Loss after epoch=24: 7.348846
Loss after epoch=25: 7.324013
Loss after epoch=26: 7.299232
Loss after epoch=27: 7.274504
Loss after epoch=28: 7.249828
Loss after epoch=29: 7.225207
Loss after epoch=30: 7.200639
Loss after epoch=3